In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
import pathlib
import json
import numpy as np
from ophys_etl.modules.decrosstalk.ophys_plane import OphysROI, OphysMovie

In [ ]:
data_dir = pathlib.Path('/Users/scott.daniel/Pika/deep_interpolation/data')
assert data_dir.is_dir()

In [ ]:
movie_path = data_dir / 'denoised.h5'
roi_path = data_dir / 'denoised_segmentation_output.json'
assert movie_path.is_file()
assert roi_path.is_file()

In [ ]:
with open(roi_path, 'rb') as in_file:
    raw_roi = json.load(in_file)

roi_list = []
for i_roi, roi_data in enumerate(raw_roi):
    roi = OphysROI(height=roi_data['height'],
                   width=roi_data['width'],
                   mask_matrix=roi_data['mask_matrix'],
                   x0=roi_data['x'],
                   y0=roi_data['y'],
                   roi_id=i_roi, valid_roi=True)
    roi_list.append(roi)

In [ ]:
ophys_movie = OphysMovie(str(movie_path.absolute()),
                         motion_border={'x0':0, 'x1':0, 'y0':0, 'y1':0})

In [ ]:
%%time
ophys_movie.load_movie_data()

In [ ]:
%%time
max_img = ophys_movie._data.max(axis=0)

In [ ]:
fig = plt.figure(figsize=(20,20))
plt.imshow(max_img, cmap='gray')

In [ ]:
max_img_rgb = np.zeros((512,512,3), dtype=int)
imax = max_img.max()
for ic in range(3):
    max_img_rgb[:, :, ic] = np.round(255*(max_img[:,:]/imax)).astype(int)

In [ ]:
fig = plt.figure(figsize=(20,20))
plt.imshow(max_img_rgb)

In [ ]:
alpha = 0.5
for roi in roi_list:
    bdry = roi.boundary_mask
    mask = roi.mask_matrix
    for icol in range(roi.width):
        for irow in range(roi.height):
            ic = None
            if bdry[irow, icol]:
                ic = 0
            elif mask[irow, icol]:
                continue
            else:
                continue
            orig = max_img_rgb[irow,icol,0]
            xx = roi.x0+icol
            yy = roi.y0+irow
            #print(max_img_rgb[yy,xx,:])
            #print(np.round(max_img_rgb[yy,xx,:]*alpha).astype(int))
            max_img_rgb[yy,xx,:] = np.round(max_img_rgb[yy,xx,:]*(1.0-alpha)).astype(int)
            max_img_rgb[yy,xx,ic] += np.round((alpha*255)).astype(int)
            
max_img_rgb = np.where(max_img_rgb<255, max_img_rgb, 255)

In [ ]:
plt.figure(figsize=(20,20))
plt.imshow(max_img_rgb)

In [ ]:
y = np.max(z, axis=0)


In [ ]:
y.shape

In [ ]:
y[2,1]

In [ ]:
z[:,2,1].max()